In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Missing Value Imputation to Nearest Bucket Value

##### This code is used to impute missing to the mean of a bucket whose target variable rate is closest to the Missing bukcet (see the chart below)

<img src="PNG/Missing2Nearest.PNG">

In [10]:
def nearest_bin(input_file, impute_column, target_variable, num_bins):
    
# Parameter Instruction:
    # input_file: the dataset contains imputed column and target variable; should be dataframe format
    # impute_column: the column with missing value, which needs imputed
    # target_variable: the column represents target variable/response variable
    # num_bins: number of buckets in calculating target rate of each bucket
    
    na_df = input_file[input_file[impute_column].isnull()]
    normal_df = df.drop(na_df.index)
    
    binned, edges = pd.cut(normal_df[impute_column], num_bins, retbins=True)
    grouped_data = list(normal_df.groupby(binned)[target_variable].mean())
    target_mean = na_df[target_variable].mean()
    bin_centers = (edges[:-1] + edges[1:])/2
    
    nearest = min(grouped_data, key=lambda x: abs(x-target_mean))
    return round(bin_centers[grouped_data.index(nearest)])

def missing2nearest(input_file, impute_column, target_variable, num_bins):
    nearest_value = nearest_bin(input_file, impute_column, target_variable, num_bins)
    input_file = input_file.fillna(nearest_value)
    return input_file

##### Example 1: Single missing cell

In [6]:
np.random.seed(1)
df = pd.DataFrame(np.random.randint(low=0, high=10, size=(4,2)),
                 columns = ['Impute_Column (X)','Target_Variable (Y)'])
df['Impute_Column (X)'][0] = None
df

,Impute_Column (X),Target_Variable (Y)
0,NaN,8
1,9.0,5
2,0.0,0
3,1.0,7


This sample data has two variables:
    1. Impute column (X), which has missing value we'd like to impute
    2. Target Variable (Y)

The target variable of that missing bucket is 8, which is closest to 7 (among 0, 5, 7) in this sample dataset, so we impute missing to the same value of X, whose corresponding Y is 7. Namely, we impute missing to 1 in this case.

In [11]:
missing2nearest(input_file = df
               ,impute_column = 'Impute_Column (X)'
               ,target_variable = 'Target_Variable (Y)'
               ,num_bins = 10)

,Impute_Column (X),Target_Variable (Y)
0,1.0,8
1,9.0,5
2,0.0,0
3,1.0,7


##### Example 2: Multiple missing cells

In [12]:
np.random.seed(1)
df = pd.DataFrame(np.random.randint(low=0, high=10, size=(10,2)),
                 columns = ['Impute_Column (X)','Target_Variable (Y)'])
df['Impute_Column (X)'][0:2] = None
df

,Impute_Column (X),Target_Variable (Y)
0,NaN,8
1,NaN,5
2,0.0,0
3,1.0,7
4,6.0,9
5,2.0,4
6,5.0,2
7,4.0,2
8,4.0,7
9,7.0,9


This sample data has two variables:
    1. Impute column (X), which has missing value we'd like to impute
    2. Target Variable (Y)

The target variable of that missing bucket is 6.5, i.e. (8+5)/2.
The non-missing buckets has following target rates:
#####    X Y
#####    0 0
#####    1 7
#####    2 4
#####    4 4.5 (i.e. (2+7)/2)
#####    5 2
#####    6 9
#####    7 9

Therefore, the target rate of missing bucket is closest to 7 (among 0, 2, 4, 4.5, 7, 9) in this sample dataset, so we impute missing to the same value of X, whose corresponding Y is 7. Namely, we impute missing to 1 in this case.

In [13]:
missing2nearest(input_file = df
               ,impute_column = 'Impute_Column (X)'
               ,target_variable = 'Target_Variable (Y)'
               ,num_bins = 10)

,Impute_Column (X),Target_Variable (Y)
0,1.0,8
1,1.0,5
2,0.0,0
3,1.0,7
4,6.0,9
5,2.0,4
6,5.0,2
7,4.0,2
8,4.0,7
9,7.0,9
